In [8]:
import geopy
import folium
import pandas as pd
import time
import numpy as np
from geopy.exc import GeocoderTimedOut
from IPython.display import clear_output

In [9]:
#replace the api_key with your own, can be registered for free at mapquest.com
geolocator = geopy.geocoders.OpenMapQuest(api_key="YOUR KEY GOES HERE")

#defines a recursive function to keep trying
# in the event of a timeout. possible improvements here:
# 1. set a certain number of tries before breaking, setting goodOrBad to 0.
def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        time.sleep(1)
        return do_geocode(address)

In [10]:
#file should be stored in xlsx (Excel data)
df = pd.read_excel(r'test_data.xlsx')
#df.head()

In [17]:
#iterate through the members list, appending two new columns for the 
# latitude and longitude of each member
addresses = df['Address_Line_1']
zipcodes = df['Zip']
city = df['City']
seriesLength = len(df['Zip'])
latitudeSer = pd.Series(np.zeros(seriesLength),index=df.index)
longitudeSer = pd.Series(np.zeros(seriesLength),index=df.index)
#next series is to report whether the address look up failed
goodOrBad = pd.Series(np.zeros(seriesLength,dtype=bool),index=df.index)

In [39]:
#Do not run this each time! Only run it upon receiving a new data set
# Otherwise, use just the map plotting tool

#probably going to need to do a considerable amount of data clean up
# 1. get rid of apartment numbers
# get rid of drive, street, et cetera
for j, address in enumerate(addresses):
    #do some text processing first
    #only take string to comma, avoiding apartment #s
    #cast to a string first
    addressCheck = str(addresses[j])
    #run through some checks:
    # 1. is there a comma? if so, only count string before it
    # 2. does the string include APT, Apt. or Apt? is so,
    #      only use string up to that point
    if addressCheck.find(',') != -1:
        addressCheck = addressCheck[1:addressCheck.find(',')]
    if addressCheck.lower().find('apt') != -1:
        addressCheck = addressCheck[1:addressCheck.lower().find('apt')]
    if addressCheck.find('#') != -1:
        addressCheck = addressCheck[1:addressCheck.find('#')]
    addressNow = addressCheck
    addressNow = addressNow.replace(".", '')
    
    #get the location
    time.sleep(1.1)
    clear_output(wait=True)
    location = do_geocode(addressNow + ',' + city[j] + ',' + zipcodes[j])
    if location == None:
        GoodToGo = False
    else:
        GoodToGo = True
        locationCoords = (location.latitude, location.longitude)
        goodOrBad[j] = 1
    #first check, change to address + zipcodes
    if GoodToGo == False:
        #try removing the city
        newAddress = addressNow + ', ' + zipcodes[j]
        time.sleep(1.0)
        location = do_geocode(newAddress)
        if location == None:
            GoodToGo = False
        else:
            GoodToGo = True
            locationCoords = (location.latitude, location.longitude)
            goodOrBad[j] = 1
    #second check, try just zipcode
    if GoodToGo == False:
        time.sleep(1.0)
        location = do_geocode(zipcodes[j])
        if location == None:
            GoodToGo = False
        else:
            GoodToGo = True
            locationCoords = (location.latitude, location.longitude)
            goodOrBad[j] = 1
    #third check, first 5 of zip-code
    if GoodToGo == False:
        time.sleep(1.0)
        location = do_geocode(zipcodes[j][0:5])
        if location == None:
            GoodToGo = False
        else:
            GoodToGo = True
            locationCoords = (location.latitude, location.longitude)
            goodOrBad[j] = 1
    #final case, just set to Ann Arbor coords
    if GoodToGo == False:
        time.sleep(1.0)
        location = do_geocode('Ann Arbor, MI')
        locationCoords = (location.latitude, location.longitude)
        goodOrBad[j] = 0
        
    print(addressNow + ',' + city[j] + ',' + zipcodes[j] + str(locationCoords))
    print('Percentage finished: ' + str(j/seriesLength*100) + '%')
    print('Successful lookups: ' + str(100*goodOrBad[0:j].mean()) + '%')
    latitudeSer[j] = location.latitude
    longitudeSer[j] = location.longitude
df['latitude'] = latitudeSer
df['longitude'] = longitudeSer
df['goodOrBad'] = goodOrBad

2951 Renfrew Street,Ann arbor,48105-1453(42.3078735, -83.7054294)
Percentage finished: 99.6845425867508%
Successful lookups: 100.0%


In [40]:
print('Valid locations found for ' + str(100*df['goodOrBad'].mean()) +'% of membership.')
#df.head()
#export the data to a .xlsx file
df.to_excel('MembersListWithCoords.xlsx')

Valid locations found for 100.0% of membership.


In [41]:
#define the map here, add the latitude and longitude points, with names and addresses
#only plot out the points that are TRUE in goodOrBad column
AnnArborCoords = do_geocode('Ann Arbor, MI')
m = folium.Map(location=[AnnArborCoords.latitude, AnnArborCoords.longitude])

validation = df['goodOrBad']
#add a circle for each point in the members list
for j, address in enumerate(df['Address_Line_1']):
      if validation[j] == True:
            #could be modified to include phone number
            toPrint = df['first_name'][j] + ' ' + df['last_name'][j] 
            folium.CircleMarker(
            popup=toPrint,
            radius=3,
            location=(df['latitude'][j], df['longitude'][j]),
            color = '#ff0000',
            fill=True,
            fill_color = "#ff0000#"
            ).add_to(m)
m

In [38]:
location = do_geocode('49240-9391')
print(location.latitude)

45.98041985


None
